In [1]:

import torch
import torch.optim as optim
import torch.nn as nn
from parse_data import get_data, get_modified_values, get_binary_values, make_data_scalar
import numpy as np
import random
from data_gen import Datagen

from generator import Generator
from evaluation import evaluate_model, bin_plot
from recognition import Recognition
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device=None
print("Using device: ", device)

import torch
print(torch.__version__)
 

Using device:  cuda
2.2.2


In [2]:
gen = Datagen(device)

x, y, x_1 = gen.get_generated_data(seq_len=2)

print("x", x.size())
print("y", y)


/home/johaanto/work/first_paper/dlgm/data_gen.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  m = df.max()[0]


x torch.Size([7652, 2])
y tensor([[1.0000],
        [0.4600],
        [1.0000],
        ...,
        [1.0000],
        [1.0000],
        [1.0000]], device='cuda:0')


In [3]:
import random

# Hyperparameters
sequence_length = [2*i for i in range(4,16)]  # 2-20 increments of two
hidden_layers = [1,2] # 1 and 2
hidden_1 = [2**i for i in range(2,7)] # 2^4 to 2^9
hidden_2 =[2**i for i in range(5,10)] # 2^2 to 2^5
variance = [0.001, 0.01, 0.005, 0.05]
lr = [0.001, 0.01, 0.1, 0.005] # stop at 0.005
data_probability = [i/5 for i in range(1,6)]
regularization = [1/i for i in range(1,10)]
for i in range(3):
    regularization.append(0)

epochs = 3500
optimizer = [optim.Adam, optim.SGD]

options = []

for seq_len in sequence_length:
    for layers in hidden_layers:
        for h1 in hidden_1:
            for h2 in hidden_2:
                for l in lr:
                    for v in variance:
                        for p in data_probability:
                            for r in regularization:
                                entry = {}
                                entry["seq_len"] = seq_len
                                entry["layers"] = layers
                                entry["latent"] = h1
                                entry["hidden"] = h2
                                entry["l"] = l
                                entry["variance"] = v
                                entry["data_prob"] = p
                                options.append(entry)
                
                                         
random.shuffle(options)    


In [ ]:
import torch.utils.data as data
from itertools import chain
import torch.nn.functional as F

def loss(y, y_hat, mean, R,g, device=None, seq_len=1):
    
    mse = nn.MSELoss().to(device)
    l = F.binary_cross_entropy(y_hat, y, reduction='sum')
    amount = mean[0].size()[0]*mean[0].size()[1]
    for m, r in zip(mean, R):
        
        C = r @ r.transpose(-2,-1) + 1e-6
        det = C.det() + 1e-6 
        l += 0.5 * torch.sum(m.pow(2).sum(-1) 
                             + C.diagonal(dim1=-2,dim2=-1).sum(-1)
                            -det.log()  -1)/amount
    
    #print(l, F.binary_cross_entropy(x_hat, x, reduction='sum'))
    return l 

best_model = None
best_score = 10000000000000000
batch_size = 10
best_history= [0,0,0,0,0,0]
history = []
history.append([1000,1000,1000])
for entry in options:

    x_d, y_d, x_d_1 = gen.get_generated_data(entry["seq_len"], entry["variance"], entry["data_prob"])
    x_t, y_t, x_t_1 = gen.get_true_data(entry["seq_len"])
    x_val, y_val, x_val_1 = gen.get_test_data(entry["seq_len"])

    print(x_d.size(), x_1.size())

    model_g = Generator(hidden_size=entry["hidden"],
                        latent_dim=entry["latent"],
                        output_dim=y_d.size()[1],
                        layers=entry["layers"],
                        seq_len=entry["seq_len"],
                        device=device)
    model_r = Recognition(input_dim=x_d.size()[1],
                          latent_dim=entry["latent"],
                          layers=entry["layers"],
                          device=device)

    loader = data.DataLoader(data.TensorDataset(x_d_1, y_d), batch_size=batch_size, shuffle=True)
    optimizer = optim.Adam(chain(model_r.parameters(), model_g.parameters()), lr=entry["l"])
    #optimizer = optim.Adam(model_r.parameters())
    bce = nn.BCELoss().to(device)
    for e in range(epochs):
        model_g.train()
        model_r.train()

        for x, y in loader:

            x.to(device)
            y.to(device)
            if x.size()[0] < batch_size:
                continue
            if random.random() < 0.5:
                continue
            rec = model_r(x)
            model_g.set_xi(rec[-1])
            y_hat = model_g()
            l = loss(y, y_hat, rec[0], rec[1], device, entry["seq_len"])
            optimizer.zero_grad()
            l.backward()
            optimizer.step()

                        

        
        if e % 10 != 0:
            continue
        
        count = 0
        sum_loss = [0, 0]
        for j in range(2):
            for x, y in loader:
                model_g.eval()
                model_r.eval()
                model_g.make_xi()
                with torch.no_grad():
                    rec = model_r(x)
                    model_g.set_xi(rec[-1])
                    b = model_g()
                    l = loss(y, b, rec[0], rec[1], device, entry["seq_len"])
                    res = []
                    
                    sum_loss[j] += l.item()
                    count += 1
                    
        
        
        sum_loss[0] /= count
     
        
        history.append([e, sum_loss[0], sum_loss[1]])
        print(history[-1])

        if len(history) > 15:
            #if no real improvements are being done stop the training. 
            # but keep doing the training if the results without correctly feeding values get better
            if abs(history[-15][1] - history[-1][1]) < 0.0001:
                break
        
    
    if history[-1][1] < best_score:
        print("New best model:\nNew loss: ", history[-1], "\nOld loss:", best_history[-1], "\nHistory:" , history[-10:])
        best_model = model_g
        best_history = history
        best_score = history[-1][1]
        best_config = entry
        with torch.no_grad():
            evaluate_model(best_model,model_r, x_t_1, y_t, x_val_1,y_val,  entry)
    else:
        with torch.no_grad():
            evaluate_model(model_g,model_r,x_t_1, y_t ,x_val_1,y_val, entry)
        print("Old model still stands:\nCurrent loss: ", history[-1], "\nBest loss:", best_history[-1])
    

/home/johaanto/work/first_paper/dlgm/data_gen.py:60: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  m = df.max()[0]


torch.Size([7652, 8]) torch.Size([7652, 2])
[0, 2.07402717024477, 3172.7082607746124]
[10, 1.9241367881976594, 2946.690011739731]
[20, 1.9146051410750995, 2933.1196088790894]
[30, 1.8947304311371347, 2901.961307644844]
[40, 1.8901921249245228, 2895.257987141609]
[50, 1.8831104813605626, 2883.6901729106903]
[60, 1.8910457726869507, 2897.741122484207]
[70, 1.8838437569203639, 2887.702768087387]
[80, 1.8820951569329354, 2885.304190516472]
[90, 1.8855957800507857, 2888.8624325990677]
[100, 1.8794051885760483, 2881.181959748268]
[110, 1.8826965344948183, 2885.0898213386536]
[120, 1.8933840669477577, 2901.765383243561]
[130, 1.87786869708614, 2877.127879023552]
[140, 1.8773937040925337, 2877.2766625881195]
[150, 1.893032036235376, 2902.1068320274353]
[160, 1.8806674325590658, 2882.60416162014]
[170, 1.876634835418771, 2874.874065876007]
[180, 1.8809066529217962, 2886.3810507059097]
[190, 1.8795598571823409, 2879.3573846817017]
[200, 1.8783674970774986, 2878.7843643426895]
[210, 1.87717437417

/home/johaanto/work/first_paper/dlgm/data_gen.py:60: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  m = df.max()[0]
/home/johaanto/work/first_paper/dlgm/data_gen.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  m = df.max()[0]


torch.Size([7652, 12]) torch.Size([7652, 2])
[0, 1.8297031677422884, 2805.785735964775]
[10, 1.7738446429099488, 2718.4900257587433]
[20, 1.6615806979081027, 2546.6760860681534]
[30, 2.0582772474687343, 3150.597635984421]
[40, 1.6597642574079976, 2543.481452226639]
[50, 1.7772811371881718, 2720.8303405046463]
[60, 1.7091696937775176, 2618.383538007736]
[70, 1.6496346687056689, 2527.7292519807816]
[80, 1.7028710558893787, 2605.984950900078]
[90, 1.6696855969746494, 2557.457218170166]
[100, 1.8776804150395854, 2867.6040024757385]
[110, 1.7197938689052592, 2637.304933309555]
[120, 1.6676356443070244, 2554.2898614406586]
[130, 1.7198152074459017, 2638.5825510025024]
[140, 1.6884332030000013, 2588.3099600076675]
[150, 1.7247677492877522, 2644.628331899643]
[160, 1.6580894886669229, 2539.2467703819275]
[170, 1.6684711696896166, 2556.106503367424]
[180, 1.6483616313797376, 2525.039198040962]
[190, 1.6492621915769952, 2528.7968468666077]
[200, 1.7196645234023311, 2636.478556394577]
[210, 1.650

In [ ]:
      
x_val, y_val, x_val_1 = gen.get_test_data(entry["seq_len"])

if history[-1][1] < best_score:
    print("New best model:\nNew loss: ", history[-1], "\nOld loss:", best_history[-1], "\nHistory:" , history[-10:])
    best_model = model_g
    best_history = history
    best_score = history[-1][1]
    best_config = entry
    with torch.no_grad():
        evaluate_model(best_model,model_r, x_t_1, y_t, x_val_1,y_val,  entry)
else:
    with torch.no_grad():
        evaluate_model(model_g,model_r,x_t_1, y_t ,x_val_1,y_val, entry)
    print("Old model still stands:\nCurrent loss: ", history[-1], "\nBest loss:", best_history[-1])
    

In [ ]:
a[:,-1,:].size()

In [ ]:
b = torch.zeros(30,1)
c = torch.zeros(1,1)


In [ ]:
torch.cat((b[1:],c))

In [ ]:
23